# Extrair apenas eventos relacionados a votos e metadados

Como o Log das urnas compila todo o histórico de eventos que ocorreu em uma unna, incluindo testes, carga, preparação, etc, para garantir uma consulta mais eficiente, são extraídos apenas os eventos relacionados aos votos em si.



## Importing libraries

In [62]:
import duckdb
import pandas as pd
import time

## Importing Data

In [63]:
DATASET = 'ALL_UFS.parquet'

In [64]:
cursor = duckdb.connect()

## Preaparing Data

### Definindo os filtros

In [66]:
COLUMN_EVENT_DESCRIPTION = 'event_description'

METADATA = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Zona Eleitoral%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Seção Eleitoral%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Município%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Local de Votação%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Turno da UE%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Identificação do Modelo de Urna%'"
]

EVENTS_DESCRIPTIONS = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Urna pronta para receber vot%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Voto confirmado par%'"
]

VOTES_DESCRIPTIONS = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Aguardando digitação do títul%'",

    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi habilitad%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi suspenso e não confirmou nenhum vot%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi suspenso pelo mesári%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor impedid%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor já justifico%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor não está votand%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor não tem idade mínim%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor sem atividade por%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor voltou a digitar enquanto estava sendo suspens%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor votou parcialmente e em seguida foi suspens%'",
    F"{COLUMN_EVENT_DESCRIPTION} = 'O voto do eleitor foi computado'",
    # 'Digital' aparece no texto
    # Dedo reconhecido no começo
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE '%Digital%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Dedo reconhecido%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Solicita digital%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Solicitação de dado pessoal do eleitor%'",
]

ACCEPTED_DATES = [
    '2022-10-03', '2022-10-30', # Constitutional Election Dates
    '2022-10-04', '2022-10-31', # In case a section lasts until the next day
]

ALL_FILTERS = METADATA + EVENTS_DESCRIPTIONS + VOTES_DESCRIPTIONS

### Construindo e executando a query

Nome do arquivo o00407-6100003300017_new.csv

SPLIT_PART(SPLIT_PART(filename, '/', 5), '-', 2)

Query the 5 first digits are the CITY CODE

Query the 4 digits after the 5 first digits are the ZONE CODE

Query the 4 digits after the 9 first digits are the SECTION CODE

In [67]:
query = F"""
    SELECT 
        *
    FROM (
        SELECT
            event_timestamp,
            event_timestamp::date AS event_date,
            event_type,
            some_id,
            event_system,
            event_description,
            event_id,
                
            REPLACE(SPLIT_PART(filename, '/', 5), '_new.csv', '') AS filename,
            
            -- Metadata from filename
            SUBSTRING( SPLIT_PART(SPLIT_PART(filename, '/', 5), '-', 2),  1, 5 ) AS city_code,
            SUBSTRING( SPLIT_PART(SPLIT_PART(filename, '/', 5), '-', 2),  6, 4 ) AS zone_code,
            SUBSTRING( SPLIT_PART(SPLIT_PART(filename, '/', 5), '-', 2), 10, 4 ) AS section_code,
            REPLACE(SPLIT_PART(filename, '/', 4), '2_', '') AS uf
        FROM
            {DATASET}
        WHERE 1=1
            AND ( {' OR '.join(ALL_FILTERS)} )
    ) _
    WHERE 1=1
    AND event_date IN ({', '.join([F"'{date}'" for date in ACCEPTED_DATES])})
"""

In [68]:
query = F"""
    COPY ({query}) TO 'UFS_VOTE_EVENTS.parquet' (FORMAT 'parquet', PARTITION_BY (event_date, uf))
"""

In [69]:
tic = time.time()
cursor.execute(query)
toc = time.time()

print(F"Query executed in {toc - tic:.2f} seconds")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Query executed in 781.79 seconds
